## **Simulation Set-Up**

In [1]:
import os
import importlib
import sys

import numpy as np
from pydrake.all import (
    AbstractValue,
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    PiecewisePose,
    ProcessModelDirectives,
    RigidTransform,
    RotationMatrix,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    LeafSystem,
    ConstantVectorSource,
    MultibodyPlant,
    Frame,
    DifferentialInverseKinematicsIntegrator,
    StateInterpolatorWithDiscreteDerivative,
    DifferentialInverseKinematicsParameters,
    LinearConstraint,
)
from IPython.display import display, SVG
import pydot
import matplotlib.pyplot as plt
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import JointStiffnessController
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation, MakeMultibodyPlant
from manipulation.utils import ConfigureParser
from manipulation.systems import AddIiwaDifferentialIK
from manipulation.meshcat_utils import AddMeshcatTriad

sys.path.append('.')
import env_ingredient_add
importlib.reload(env_ingredient_add)


<module 'env_ingredient_add' from '/home/nep/robot_locomotion/manipulation/project/env_ingredient_add.py'>

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Simulation Set Up 

Generating a useful simulation scene is the first step in successful manipulation projects. In this exercise, we will learn how to add the necessary objects to set-up a scene.

**These are the learning goals of this exercise:**
1. Be able to work with objects written in robot description formats such as URDFs and SDFs.
2. Understand how to view and edit collision geometries
3. Understand how to add objects to Drake to create a useful manipulation scene.

A great resource is [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Parts of this notebook were taken from it.

## Simulated Kitchen Setup & Robot Configuration
In the included `objects/environment_setup.yaml` file, we instantiate all the parts of the kitchen scene.

In [3]:
if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
    #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
    scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
    xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
else:
    #scene = open("objects/environment_setup.yaml") # local setup
    scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
    xmls = [os.getcwd() + "/package.xml"]

scenario = LoadScenario(data=scene)

In [4]:
scenario

Scenario(random_seed=0, simulation_duration=inf, simulator_config=SimulatorConfig(integration_scheme='runge_kutta3', max_step_size=0.01, accuracy=0.01, use_error_control=False, target_realtime_rate=0.0, publish_every_time_step=False), plant_config=MultibodyPlantConfig(time_step=0.001, use_sampled_output_ports=True, penetration_allowance=0.001, stiction_tolerance=0.001, contact_model='hydroelastic_with_fallback', discrete_contact_approximation='sap', discrete_contact_solver='', sap_near_rigid_threshold=1.0, contact_surface_representation='polygon', adjacent_bodies_collision_filters=True), directives=[ModelDirective(add_model=AddModel(file='package://manipulation/mobile_iiwa14_primitive_collision.urdf', name='mobile_iiwa', default_joint_positions={'iiwa_base_x': array([0.]), 'iiwa_base_y': array([0.]), 'iiwa_base_z': array([0.]), 'iiwa_joint_1': array([0.]), 'iiwa_joint_2': array([0.1]), 'iiwa_joint_3': array([0.]), 'iiwa_joint_4': array([-1.2]), 'iiwa_joint_5': array([0.]), 'iiwa_joint_

In [ ]:
class PoseTrajectorySource(LeafSystem):
    """
    returns desired list of poses of dimension 20: 10 positions, 10 velocities
    (optional) pose_trajectory: trajectory to follow. if context does not already exist, pass it in from the plant.
    """
    # pose_trajectory: PiecewisePose = PiecewisePose()

    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))
        pose = self._pose_trajectory.GetPose(context.get_time())
        # print(f"Pose dimensions: {pose.GetAsVector().size()}")
        output.set_value(pose)

def CreateIiwaControllerPlant():
    #creates plant that includes only the robot and gripper, used for controllers
    robot_sdf_path = ("package://manipulation/mobile_iiwa14_primitive_collision.urdf")
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    ConfigureParser(parser)
    parser.AddModelsFromUrl(robot_sdf_path)
    
    # Add gripper.
    parser.AddModelsFromUrl("package://manipulation/schunk_wsg_50_welded_fingers.dmd.yaml")
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_M=plant_robot.GetFrameByName("body"),
        X_FM=RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 0, 0.09])
    )
    
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()
    return plant_robot


def AddMobileIiwaDifferentialIK(
    builder: DiagramBuilder, plant: MultibodyPlant, frame: Frame = None
) -> DifferentialInverseKinematicsIntegrator:
    """
    Args:
        builder: The DiagramBuilder to which the system should be added.

        plant: The MultibodyPlant passed to the DifferentialInverseKinematicsIntegrator.

        frame: The frame to use for the end effector command. Defaults to the body
            frame of "iiwa_link_7". NOTE: This must be present in the controller plant!

    Returns:
        The DifferentialInverseKinematicsIntegrator system.
    """
    assert plant.num_positions() == 10
    
    params = DifferentialInverseKinematicsParameters(
        plant.num_positions(), plant.num_velocities()   
    )
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(2)
    params.set_end_effector_translational_velocity_limits([-2, -2, -2], [2, 2, 2])

    if frame is None:
        frame = plant.GetFrameByName("iiwa_link_7")
        

    mobile_iiwa_velocity_limits = np.array([0.5, 0.5, 0.5, 1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
    params.set_joint_velocity_limits(
        (-mobile_iiwa_velocity_limits, mobile_iiwa_velocity_limits)
    )
    params.set_joint_centering_gain(10 * np.eye(10))
    
            
    differential_ik = builder.AddNamedSystem(
        "diff_ik_integrator",
        DifferentialInverseKinematicsIntegrator(
            plant,
            frame,
            time_step,
            params,
            log_only_when_result_state_changes=True,
        )
    )
    return differential_ik

class PizzaBot:
    # make robot station and set up simulation
    def __init__(self, scenario=None, traj=PiecewisePose()):
        self.meshcat = meshcat
        builder = DiagramBuilder()
        
        self.station = builder.AddSystem(MakeHardwareStation(
            scenario, meshcat, package_xmls=[os.getcwd() + "/package.xml"])
        )
        
        # add plant (iiwa arm)
        self.plant = self.station.GetSubsystemByName("plant")
        # self.plant.GetJointByName("iiwa_joint_7").set_position_limits(
        #     [-np.inf], [np.inf]
        # )
        
        # self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        # self.controller = self.station.GetSubsystemByName("mobile_iiwa.controller")


        # this is the workaround. right now we don't have the InverseDynamicsController hooked up.
        self.iiwa_controller_plant = CreateIiwaControllerPlant()  
        self.controller = AddMobileIiwaDifferentialIK(
            builder,
            plant=self.iiwa_controller_plant,
            frame=self.gripper_frame,
        )
        # NOTE: This controller plant creation requires https://github.com/RussTedrake/manipulation/pull/385
        # A workaround for now is to create a plant that just contains the mobile iiwa. This should be easy
        # with the existing `robot_only` directive string. This wouldn't use MakeHardwareStation but would
        # use `parser.AddDirectives` with `MultibodyPlant`. See https://github.com/RussTedrake/manipulation/pull/371
        # if this is unclear.
    
        if traj is not None:
            self.traj_source = builder.AddSystem(PoseTrajectorySource(traj))
            # print(self.traj_source)

            # TODO(mjg): do the workaround

        else:
            assert False, "Need to provide a trajectory"
        
        builder.Connect(
            self.traj_source.get_output_port(),
            self.controller.get_input_port(0),
        )
        # If bug, see https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_differential_inverse_kinematics_integrator.html for other ports or set positions commaned
        
        num_positions = 10
        time_step = 1e-3 # TODO: the timestep you use
        pos_to_state_sys = builder.AddSystem(
                StateInterpolatorWithDiscreteDerivative(
                    num_positions,
                    time_step,
                    suppress_initial_transient=True,
                )
            )
        
        builder.Connect(
            self.station.GetOutputPort("mobile_iiwa.state_estimated"),
            self.controller.GetInputPort("robot_state"),
        )
        
        builder.Connect(
            self.controller.get_output_port(),
            pos_to_state_sys.get_input_port(),
        )
        
        builder.Connect(
            pos_to_state_sys.get_output_port(),
            self.station.GetInputPort("mobile_iiwa.desired_state")
        )

        # builder.ExportOutput(
        #     self.station.GetOutputPort("mobile_iiwa.state_estimated"),
        #     "mobile_iiwa.state_estimated"
        # )

        self.diagram = builder.Build() # IMPT: must build the diagram before visualizing it
        self.diagram.set_name("diagram")

        self.simulator = Simulator(self.diagram)
        


    def print_diagram(self, output_file="diagram.png"):
        print("station.system names: ")
        for sys in self.station.GetSystems():
            print("- " + sys.GetSystemName())

        # Visualize and save the diagram as a PNG
        graph = pydot.graph_from_dot_data(
            self.diagram.GetGraphvizString(max_depth=10)
        )[0]
        graph.write_png(output_file)
        print(f"Diagram saved to {output_file}")


    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()

        # # provide initial states
        # q0 = np.array(
        #     [
        #         1.40666193e-03,
        #         1.56461165e-01,
        #         -3.82761069e-03,
        #         -1.32296976e00,
        #         -6.29097287e-03,
        #         1.61181157e00,
        #         -2.66900985e-03,
        #         -1,
        #         1,
        #         0
        #     ]
        # )
        v0 = np.zeros(10)
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("mobile_iiwa")
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        # self.plant.SetPositions(plant_context, iiwa, q0)
        # self.plant.SetVelocities(plant_context, iiwa, v0)

        gripper = self.plant.GetModelInstanceByName("gripper")
        
        # self.plant.SetPositions(plant_context, gripper, [-0.05, 0.05])
        # self.plant.SetVelocities(plant_context, gripper, [0, 0])

        return context



    # run simulation
    def run_simulation(self, start_time):
        # context = self.simulator.get_mutable_context()
        # x0 = self.station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
        # self.station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
        self.meshcat.StartRecording()
        self.simulator.AdvanceTo(start_time if running_as_notebook else 0.1)
        self.meshcat.PublishRecording()
    

def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=xmls
    )
    simulator = Simulator(station)
    context = simulator.get_mutable_context()
    x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
    station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()




In [6]:
bot = PizzaBot(scenario)
bot.print_diagram()

station.system names: 
- plant
- scene_graph
- DrakeLcm(bus_name=default)
- DrakeLcm(bus_name=opt_out)
- mobile_iiwa.controller
- meshcat_visualizer(illustration)
- meshcat_visualizer(inertia)
- meshcat_visualizer(proximity)
- meshcat_contact_visualizer
- inertia_visualizer
Diagram saved to diagram.png


In [7]:
# default_joint_angles = np.array([
#     1.40666193e-05,
#     1.56461165e-01,
#     -3.82761069e-05,
#     -1.32296976e00,
#     -6.29097287e-06,
#     1.61181157e00,
#     -2.66900985e-05,
#     0,
#     0,
#     0
# ])

# bot = PizzaBot(scenario)
# # bot.run_simulation(start_time=2.0)

# bot.print_diagram()

In [8]:
# define center and radius
radius = 0.1
p0 = [0.45, 0.0, 0.4]
p_base = [0.0, 0.0, 0.0]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WCenter = RigidTransform(R0, p0)

num_key_frames = 10
"""
you may use different thetas as long as your trajectory starts
from the Start Frame above and your rotation is positive
in the world frame about +z axis
thetas = np.linspace(0, 2*np.pi, num_key_frames)
"""
thetas = np.linspace(0, 2 * np.pi, num_key_frames)

# bot = PizzaBot(scenario)

def compose_circular_key_frames(thetas, X_WCenter, radius):
    """
    returns: a list of RigidTransforms
    """
    # this is an template, replace your code below
    key_frame_poses_in_world = []
    for theta in thetas:
        position = X_WCenter.translation() + np.array([
            radius * np.cos(theta),
            radius * np.sin(theta),
            0.0  # z-coordinate stays constant for a horizontal trajectory
        ])
        
        # Use the same rotation matrix for all keyframes
        rotation_matrix = X_WCenter.rotation()
        this_pose = RigidTransform(rotation_matrix, position)
        key_frame_poses_in_world.append(this_pose)
        # print(f"Key frame pose at theta {theta}: Position = {position}, Rotation = {rotation_matrix}")

    return key_frame_poses_in_world


key_frame_poses = compose_circular_key_frames(thetas, X_WCenter, radius)
print("key_frame_poses: ", key_frame_poses)
# print("length of key_frame_poses: ", len(key_frame_poses))

X_WGinit = bot.get_X_WG(context=bot.diagram.CreateDefaultContext())
print("X_WGinit: ", X_WGinit)

total_time = 20.0
start_time = total_time 
key_frame_poses = [X_WGinit] + compose_circular_key_frames(thetas, X_WCenter, radius)

for i, frame in enumerate(key_frame_poses):
    AddMeshcatTriad(meshcat, X_PT=frame, path="frame"+str(i))

times = np.linspace(0, total_time, num_key_frames + 1)

traj = PiecewisePose.MakeLinear(times, key_frame_poses)

X_WGinit1 = RigidTransform(R0, p0 - np.array([0.0, 0.0, 0.1]))

traj_bot = PizzaBot(scenario, traj=traj)
print("Starting simulation")
traj_bot.run_simulation(start_time=start_time)

key_frame_poses:  [RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 0.0, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, 0.06427876096865393, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, 0.0984807753012208, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.4, 0.08660254037844388, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, 0.03420201433256689, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, -0.034202014332566866, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.

In [9]:
bot.print_diagram()
# context = bot.station.GetMyContextFromRoot(bot.diagram.CreateDefaultContext())
# bot.controller.GetInputPort("desired_state").FixValue(context, default_joint_angles)



station.system names: 
- plant
- scene_graph
- DrakeLcm(bus_name=default)
- DrakeLcm(bus_name=opt_out)
- mobile_iiwa.controller
- meshcat_visualizer(illustration)
- meshcat_visualizer(inertia)
- meshcat_visualizer(proximity)
- meshcat_contact_visualizer
- inertia_visualizer
Diagram saved to diagram.png


In [10]:
# !pwd

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>